In [14]:
from glob import glob
import pandas as pd

# Reading data from our CSV files that are stored inside this repository (single files)

try: 
    single_file_accelerometer = pd.read_csv(
        '../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv')
    single_file_gyroscope = pd.read_csv(
        '../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv')
except FileNotFoundError:
    print("Files that you try to use are not available.")

# Reading all CVS files from data/raw/MetaMotion that we will use as a list later 

# Read all files that have csv extension if inside this repo the compiler finds some files that are not with this extension they 
# will be ignored

files = glob('../../data/raw/MetaMotion/*.csv')

# Extract pieces of the file name e.g. A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv and append 
# it to the data frame

data_path = '../../data/raw/MetaMotion/'

first_file = files[0]

# We will extract three variables from the file name: participant: A, exercise: bench, category of sets(e.g. heavy)

participant = first_file.split('-')[0].replace(data_path,'') # on this way we are updating existing path and getting the participate
exercise = first_file.split('-')[1]
category = first_file.split('-')[2].rstrip('123') # with category we have the number of the set so we need to remove it

data_frame = pd.read_csv(first_file)

data_frame['participant'] = participant # adding new columns inside the data frame
data_frame['exercise'] = exercise
data_frame['category'] = category


# --------------------------------------------------------------
# Read all files
# --------------------------------------------------------------


# --------------------------------------------------------------
# Working with datetimes
# --------------------------------------------------------------


# --------------------------------------------------------------
# Turn into function
# --------------------------------------------------------------


# --------------------------------------------------------------
# Merging datasets
# --------------------------------------------------------------


# --------------------------------------------------------------
# Resample data (frequency conversion)
# --------------------------------------------------------------

# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz


# --------------------------------------------------------------
# Export dataset
# --------------------------------------------------------------
